##This notebook finds the organism for each uniprot ID in the database and copies data from the SQLite database tables to mySQL tables.

In [26]:
import sqlite3
import pandas as pd
import yaml
import os
import mysql.connector
import numpy as np
from Bio import Entrez, SeqIO
from io import StringIO
import requests


#Adjust parameters in the settings.yaml file

In [18]:
settings_file = open('/Users/aliceaakerberg/Sites/scripts/settings.yaml','r')
settings = yaml.load(settings_file,Loader=yaml.FullLoader)

db_path = settings[0]['database']['database_path']
username = settings[0]['database']['username']
password = settings[0]['database']['password']
host = settings[0]['database']['host']
database_name = settings[0]['database']['database_name']
output_file_path = settings[1]['output']['output_file_path']
Entrez.email = settings[2]['Entrez']['email']

##Step 1: Retrieve organism data for each uniprot ID using NCBI Entrez. (This process can take a while (500 UniProt takes approx. 10min). Entrez may prematurely terminate execution if it deems the task excessive. It is recommended to split the UniProt IDs into small groups and execute script with these groups one by one to prevent premature termination.)

In [33]:
# This code chunk checks that Entrez E-utilities is working correctly. Ensure it does not produce an error before proceeding.

url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
params = {
    "db": "protein",
    "id": "B7ZW38",
    "rettype": "gb",
    "retmode": "text"
}

response = requests.get(url, params=params, verify=False)
response.raise_for_status()  # This will raise an HTTPError if the HTTP request returned an unsuccessful status code.

handle = StringIO(response.text)
record = SeqIO.read(handle, "genbank")
handle.close()

organism = record.annotations.get('organism', 'Organism not found')
print(f'Organism: {organism}')


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eutils.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Organism: Homo sapiens


In [39]:
def get_organism(db_path):
    sqlite_conn = sqlite3.connect(db_path)
    uniprot_df = pd.read_sql(f'SELECT ID FROM processed_files_log', sqlite_conn)
    uniprot_ids = uniprot_df['ID'].tolist()
    sqlite_conn.close()

    results = []

    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"

    counter = 0

    for id in uniprot_ids:
        params = {
        "db": "protein",
        "id": id,
        "rettype": "gb",
        "retmode": "text"
        }

        response = requests.get(url, params=params, verify=False)
        response.raise_for_status()  # This will raise an HTTPError if the HTTP request returned an unsuccessful status code.

        handle = StringIO(response.text)
        record = SeqIO.read(handle, "genbank")
        handle.close()

        organism = record.annotations.get('organism', 'Organism not found')

        results.append({
            "uniprot_id": id,
            "organism": organism
        })

        counter += 1
        print("Organism fetched for UniProt ", id, "(", str(counter), " out of ", str(len(uniprot_ids)), ")")

    # Step 3: Convert the list of results into a DataFrame
    results_df = pd.DataFrame(results)

    return results_df

In [ ]:
organism_info = get_organism(db_path)

In [45]:
def insert_protein_info(df_name, protein_info_mysql_table):    
    conn = mysql.connector.connect(user=username, password=password, host=host, database=database_name)
    cursor = conn.cursor()

    cursor.execute(f"SELECT * FROM {protein_info_mysql_table}")
    existing_rows = cursor.fetchall()

    # Create a set of tuples representing existing rows for fast lookup
    existing_rows_set = set(existing_rows)

    # Step 3: Insert DataFrame into the MySQL table without duplicates
    columns = ', '.join(df_name.columns)
    placeholders = ', '.join(['%s'] * len(df_name.columns))
    insert_query = f"INSERT INTO {protein_info_mysql_table} ({columns}) VALUES ({placeholders})"

    for i, row in df_name.iterrows():
        row_tuple = tuple(None if pd.isna(val) else val for val in row)
        if row_tuple not in existing_rows_set:
            try:
                cursor.execute(insert_query, row_tuple)
            except mysql.connector.Error as err:
                print(f"Error: {err}")
                print(f"Failed to insert row: {row_tuple}")

    conn.commit()

    # Step 4: Close the cursor and connection
    cursor.close()
    conn.close()

    print(f"Data imported successfully into table {protein_info_mysql_table}!")

In [47]:
protein_info_mysql_table = "protein_info"
insert_protein_info(organism_info, protein_info_mysql_table)

Data imported successfully into table protein_info!


#Step 2: Retrieve SQlite table names from the database. These names will be used to iteratively copy data to the mySQL database.

In [ ]:
def get_table_names(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Execute a query to retrieve the table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    # Close the connection
    conn.close()

    # Store the table names in a list
    table_names = [table[0] for table in tables]
    return table_names

In [ ]:
table_names = get_table_names(db_path)
print(table_names)

#Step 3: Copy data from SQLite database to mySQL database.

In [ ]:
def transfer_sqlite_to_df(db_path, table_name, output_file_path):
    # Connect to the SQLite3 database
    sqlite_conn = sqlite3.connect(db_path)
    # Read the data from the SQLite3 table into a pandas DataFrame
    df = pd.read_sql(f'SELECT * FROM {table_name}', sqlite_conn)
    sqlite_conn.close()

    # Check for column names with whitespace and replace with underscores
    original_columns = df.columns.tolist()
    new_columns = [col.replace(' ', '_').replace('_-_', '_').replace('-', '_') for col in original_columns]

    # Print which column names have been changed
    for original, new in zip(original_columns, new_columns):
        if original != new:
            print(f"Column name changed in '{table_name}': '{original}' to '{new}'")

    df.columns = new_columns

    #Remove the # at the start of the next 4 lines to download database tables to output files directory

    #os.makedirs(output_file_path, exist_ok=True)
    #output_file_path = f"{output_file_path}/{table_name}.tsv"
    #df.to_csv(output_file_path, sep='\t', index=False)
    #print(f"Data from '{table_name}' saved to '{output_file_path}' successfully.")

    df.replace('', np.nan, inplace=True)
    df.replace({np.inf: 'inf', -np.inf: '-inf'}, inplace=True)

    conn = mysql.connector.connect(user=username, password=password, host=host, database=database_name)
    cursor = conn.cursor()

    cursor.execute(f"SELECT * FROM {table_name}")
    existing_rows = cursor.fetchall()

    # Create a set of tuples representing existing rows for fast lookup
    existing_rows_set = set(existing_rows)

    # Step 3: Insert DataFrame into the MySQL table without duplicates
    columns = ', '.join(df.columns)
    placeholders = ', '.join(['%s'] * len(df.columns))
    insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

    for i, row in df.iterrows():
        row_tuple = tuple(None if pd.isna(val) else val for val in row)
        if row_tuple not in existing_rows_set:
            try:
                cursor.execute(insert_query, row_tuple)
            except mysql.connector.Error as err:
                print(f"Error: {err}")
                print(f"Failed to insert row: {row_tuple}")

    conn.commit()

    # Step 4: Close the cursor and connection
    cursor.close()
    conn.close()

    print(f"Data imported successfully into table {table_name}!")

In [ ]:
for table in table_names:
    transfer_sqlite_to_df(db_path, table, output_file_path)

##Done!